# Lab 3: Extending Azure SRE Agent with AG-UI

**Duration:** 15 minutes  
**Objective:** Build custom SRE tools, serve them via AG-UI protocol, and see rich data rendered as UI components

---

## What You'll Learn

1. **Create SRE tools** that return structured data (metrics, incidents, runbook results)
2. **Build a real agent** using Microsoft Agent Framework with these tools
3. **Serve via AG-UI** - expose your agent through the AG-UI protocol for rich UI rendering
4. **Connect with MCP** - understand how MCP servers extend agent capabilities

---

## Prerequisites

- Completed Lab 1 (Agent Framework basics)
- `.env` file configured with `PROJECT_ENDPOINT`

---

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────┐
│                       Your SRE Agent                                │
├─────────────────────────────────────────────────────────────────────┤
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐    │
│  │  query_metrics  │  │ execute_runbook │  │   get_alerts    │    │
│  │   Returns:      │  │   Returns:      │  │   Returns:      │    │
│  │   MetricsData   │  │   RunbookResult │  │   AlertList     │    │
│  └────────┬────────┘  └────────┬────────┘  └────────┬────────┘    │
│           │                    │                    │              │
│           └────────────────────┼────────────────────┘              │
│                                │                                   │
│                    ┌───────────▼───────────┐                       │
│                    │   AG-UI Protocol      │                       │
│                    │   (FastAPI endpoint)  │                       │
│                    └───────────┬───────────┘                       │
└────────────────────────────────┼───────────────────────────────────┘
                                 │
              ┌──────────────────┼──────────────────┐
              │                  │                  │
              ▼                  ▼                  ▼
     ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
     │  CopilotKit │    │  Custom UI  │    │    CLI      │
     │   Client    │    │   Client    │    │   Client    │
     └─────────────┘    └─────────────┘    └─────────────┘
           │                  │                  │
           ▼                  ▼                  ▼
     ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
     │  📊 Charts  │    │  🎨 Cards   │    │  📝 JSON    │
     │  📋 Tables  │    │  📈 Graphs  │    │  Text       │
     └─────────────┘    └─────────────┘    └─────────────┘
```

**AG-UI** (Agent-UI Protocol) standardizes how agents communicate with UIs, enabling:
- Streaming responses
- Rich tool result rendering
- Any client (web, CLI, mobile) to connect to your agent

---

## Step 1: Install Dependencies

In [ ]:
%pip install agent-framework agent-framework-azure-ai agent-framework-ag-ui --pre --quiet
%pip install fastapi uvicorn nest-asyncio pyyaml --quiet

---

## Step 2: Define SRE Tools with Rich Structured Output

These tools return complex data structures that AG-UI clients can render as rich components.

In [ ]:
import os
import json
from typing import Annotated, Any, Dict, List
from datetime import datetime, timedelta
from pydantic import Field
from dotenv import load_dotenv

from agent_framework import ChatAgent, tool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

load_dotenv()

print("✓ Imports loaded")

In [ ]:
# Tool 1: Query service metrics - returns rich chart data
@tool
def query_service_metrics(
    service_name: Annotated[str, Field(description="Name of the service to query (e.g., 'payment-service', 'inventory-service')")],
    time_range: Annotated[str, Field(description="Time range for metrics (e.g., '1h', '6h', '24h')")] = "1h"
) -> Dict[str, Any]:
    """
    Query metrics for a service. Returns structured data suitable for charts/graphs.
    Use this to check service health, latency, error rates, and throughput.
    """
    services = {
        "payment-service": {
            "status": "degraded",
            "metrics": {
                "request_rate": {"value": 1250, "unit": "req/min", "trend": "up", "change": "+15%"},
                "error_rate": {"value": 2.5, "unit": "%", "trend": "up", "change": "+1.8%"},
                "p50_latency": {"value": 45, "unit": "ms", "trend": "stable", "change": "0%"},
                "p99_latency": {"value": 892, "unit": "ms", "trend": "up", "change": "+340%"},
                "cpu_usage": {"value": 78, "unit": "%", "trend": "up", "change": "+23%"},
                "memory_usage": {"value": 4.2, "unit": "GB", "trend": "up", "change": "+800MB"}
            },
            "time_series": [
                {"timestamp": "14:00", "latency_p99": 250, "error_rate": 0.5},
                {"timestamp": "14:15", "latency_p99": 280, "error_rate": 0.7},
                {"timestamp": "14:30", "latency_p99": 450, "error_rate": 1.2},
                {"timestamp": "14:45", "latency_p99": 720, "error_rate": 2.1},
                {"timestamp": "15:00", "latency_p99": 892, "error_rate": 2.5}
            ],
            "alerts": [
                {"severity": "critical", "message": "P99 latency exceeded 500ms threshold"},
                {"severity": "warning", "message": "Error rate above 2% for 15 minutes"}
            ]
        },
        "inventory-service": {
            "status": "healthy",
            "metrics": {
                "request_rate": {"value": 800, "unit": "req/min", "trend": "stable", "change": "+2%"},
                "error_rate": {"value": 0.1, "unit": "%", "trend": "stable", "change": "0%"},
                "p50_latency": {"value": 25, "unit": "ms", "trend": "stable", "change": "-5%"},
                "p99_latency": {"value": 180, "unit": "ms", "trend": "stable", "change": "-2%"},
                "cpu_usage": {"value": 35, "unit": "%", "trend": "stable", "change": "+1%"},
                "memory_usage": {"value": 1.8, "unit": "GB", "trend": "stable", "change": "+50MB"}
            },
            "time_series": [
                {"timestamp": "14:00", "latency_p99": 175, "error_rate": 0.1},
                {"timestamp": "14:15", "latency_p99": 178, "error_rate": 0.1},
                {"timestamp": "14:30", "latency_p99": 182, "error_rate": 0.1},
                {"timestamp": "14:45", "latency_p99": 179, "error_rate": 0.1},
                {"timestamp": "15:00", "latency_p99": 180, "error_rate": 0.1}
            ],
            "alerts": []
        }
    }
    
    service = services.get(service_name.lower())
    if not service:
        return {"error": f"Service '{service_name}' not found", "available_services": list(services.keys())}
    
    return {"service": service_name, "time_range": time_range, "queried_at": datetime.now().isoformat(), **service}

print("✓ query_service_metrics tool defined")

In [ ]:
# Tool 2: Get active alerts - returns tabular data
@tool
def get_active_alerts(
    severity_filter: Annotated[str, Field(description="Filter by severity: 'all', 'critical', 'warning', or 'info'")] = "all"
) -> Dict[str, Any]:
    """
    Get all active alerts across the infrastructure.
    Returns a list of alerts with severity, affected resources, and recommended actions.
    """
    alerts = [
        {
            "id": "ALT-001",
            "severity": "critical",
            "title": "High P99 Latency - Payment Service",
            "resource": "payment-service",
            "resource_type": "AKS Deployment",
            "metric": "p99_latency",
            "current_value": "892ms",
            "threshold": "500ms",
            "triggered_at": (datetime.now() - timedelta(minutes=18)).isoformat(),
            "duration": "18 minutes",
            "possible_causes": ["Database connection pool exhaustion", "Memory leak causing GC pauses", "Downstream service degradation"],
            "recommended_action": "Check database connections and recent deployments"
        },
        {
            "id": "ALT-002",
            "severity": "critical",
            "title": "Pod Restart Loop - Payment Service",
            "resource": "payment-service-7d4f8b9c5-x2k9j",
            "resource_type": "AKS Pod",
            "metric": "restart_count",
            "current_value": "8 restarts",
            "threshold": "3 restarts/hour",
            "triggered_at": (datetime.now() - timedelta(minutes=12)).isoformat(),
            "duration": "12 minutes",
            "possible_causes": ["OOMKilled - Memory limit exceeded", "Liveness probe failures", "Application crash on startup"],
            "recommended_action": "Check pod logs and memory usage"
        },
        {
            "id": "ALT-003",
            "severity": "warning",
            "title": "Elevated Error Rate - Payment Service",
            "resource": "payment-service",
            "resource_type": "AKS Deployment",
            "metric": "error_rate",
            "current_value": "2.5%",
            "threshold": "1%",
            "triggered_at": (datetime.now() - timedelta(minutes=15)).isoformat(),
            "duration": "15 minutes",
            "possible_causes": ["Upstream dependency failures", "Invalid request payload from clients", "Configuration drift after deployment"],
            "recommended_action": "Review error logs for specific failure patterns"
        },
        {
            "id": "ALT-004",
            "severity": "warning",
            "title": "High CPU Usage - Database Server",
            "resource": "sql-prod-001",
            "resource_type": "Azure SQL",
            "metric": "cpu_percent",
            "current_value": "78%",
            "threshold": "70%",
            "triggered_at": (datetime.now() - timedelta(minutes=25)).isoformat(),
            "duration": "25 minutes",
            "possible_causes": ["Expensive queries from payment service", "Missing index on frequently queried table", "Lock contention during peak load"],
            "recommended_action": "Review Query Performance Insights"
        }
    ]
    
    if severity_filter != "all":
        alerts = [a for a in alerts if a["severity"] == severity_filter]
    
    summary = {
        "total": len(alerts),
        "by_severity": {
            "critical": len([a for a in alerts if a["severity"] == "critical"]),
            "warning": len([a for a in alerts if a["severity"] == "warning"]),
            "info": len([a for a in alerts if a["severity"] == "info"])
        }
    }
    
    return {"summary": summary, "alerts": alerts, "queried_at": datetime.now().isoformat()}

print("✓ get_active_alerts tool defined")

In [ ]:
# Tool 3: Execute runbook - returns step-by-step execution results
@tool
def execute_runbook(
    runbook_name: Annotated[str, Field(description="Name of the runbook to execute")],
    target_resource: Annotated[str, Field(description="Resource to apply the runbook to")],
    dry_run: Annotated[bool, Field(description="If true, simulate execution without making changes")] = True
) -> Dict[str, Any]:
    """
    Execute an automation runbook for incident remediation.
    IMPORTANT: Always use dry_run=true first to preview changes before actual execution.
    """
    runbooks = {
        "restart-service": {
            "description": "Gracefully restart a service with health checks",
            "risk_level": "low",
            "estimated_duration": "2-3 minutes",
            "steps": [
                {"step": 1, "action": "Scale deployment to 0 replicas", "duration": "30s"},
                {"step": 2, "action": "Wait for graceful shutdown", "duration": "15s"},
                {"step": 3, "action": "Clear local caches", "duration": "5s"},
                {"step": 4, "action": "Scale deployment back to original count", "duration": "30s"},
                {"step": 5, "action": "Wait for pods to be ready", "duration": "60s"},
                {"step": 6, "action": "Run health check validation", "duration": "15s"}
            ],
            "rollback_available": True
        },
        "scale-out": {
            "description": "Increase replica count to handle load",
            "risk_level": "low",
            "estimated_duration": "1-2 minutes",
            "steps": [
                {"step": 1, "action": "Check current replica count", "duration": "5s"},
                {"step": 2, "action": "Verify cluster has sufficient resources", "duration": "10s"},
                {"step": 3, "action": "Increase replica count by 50%", "duration": "30s"},
                {"step": 4, "action": "Wait for new pods to be scheduled", "duration": "45s"},
                {"step": 5, "action": "Verify load balancer registration", "duration": "15s"}
            ],
            "rollback_available": True
        },
        "clear-connection-pool": {
            "description": "Reset database connection pool to clear stale connections",
            "risk_level": "medium",
            "estimated_duration": "30 seconds",
            "steps": [
                {"step": 1, "action": "Identify pods with high connection age", "duration": "10s"},
                {"step": 2, "action": "Trigger connection pool refresh via admin endpoint", "duration": "5s"},
                {"step": 3, "action": "Verify new connections established", "duration": "10s"},
                {"step": 4, "action": "Monitor query latency for improvement", "duration": "30s"}
            ],
            "rollback_available": False
        },
        "rollback-deployment": {
            "description": "Roll back to the previous stable deployment",
            "risk_level": "medium",
            "estimated_duration": "3-5 minutes",
            "steps": [
                {"step": 1, "action": "Identify previous stable revision", "duration": "10s"},
                {"step": 2, "action": "Verify previous image is available", "duration": "15s"},
                {"step": 3, "action": "Initiate rolling update to previous version", "duration": "60s"},
                {"step": 4, "action": "Monitor rollout progress", "duration": "90s"},
                {"step": 5, "action": "Run smoke tests against rolled-back version", "duration": "30s"},
                {"step": 6, "action": "Update deployment annotations", "duration": "5s"}
            ],
            "rollback_available": True
        }
    }
    
    runbook = runbooks.get(runbook_name)
    if not runbook:
        return {
            "error": f"Runbook '{runbook_name}' not found",
            "available_runbooks": [{"name": name, "description": rb["description"], "risk_level": rb["risk_level"]} for name, rb in runbooks.items()]
        }
    
    execution_result = {
        "runbook": runbook_name,
        "target_resource": target_resource,
        "mode": "DRY RUN" if dry_run else "LIVE EXECUTION",
        "description": runbook["description"],
        "risk_level": runbook["risk_level"],
        "estimated_duration": runbook["estimated_duration"],
        "rollback_available": runbook["rollback_available"],
        "execution_plan": runbook["steps"],
        "started_at": datetime.now().isoformat()
    }
    
    if dry_run:
        execution_result["status"] = "preview"
        execution_result["message"] = "This is a dry run. No changes will be made. Set dry_run=false to execute."
    else:
        execution_result["status"] = "completed"
        execution_result["execution_id"] = f"exec-{datetime.now().strftime('%Y%m%d%H%M%S')}"
        execution_result["completed_at"] = (datetime.now() + timedelta(minutes=2)).isoformat()
        execution_result["steps_completed"] = [{**step, "status": "completed", "output": "Success"} for step in runbook["steps"]]
        execution_result["message"] = f"Runbook '{runbook_name}' executed successfully on {target_resource}"
    
    return execution_result

print("✓ execute_runbook tool defined")

In [ ]:
# Tool 4: Investigate incident - returns comprehensive analysis
@tool
def investigate_incident(
    alert_id: Annotated[str, Field(description="Alert ID to investigate (e.g., 'ALT-001')")]
) -> Dict[str, Any]:
    """
    Perform root cause analysis on an alert. Correlates logs, metrics, and recent changes
    to identify the most likely cause and recommended remediation.
    """
    investigations = {
        "ALT-001": {
            "alert_id": "ALT-001",
            "title": "High P99 Latency - Payment Service",
            "investigation_status": "completed",
            "timeline": [
                {"time": "14:28:00", "event": "Deployment payment-service:v2.3.1 completed", "source": "ArgoCD"},
                {"time": "14:32:15", "event": "First slow query detected (>500ms)", "source": "APM"},
                {"time": "14:35:00", "event": "Database connection pool utilization hit 95%", "source": "Prometheus"},
                {"time": "14:38:45", "event": "P99 latency crossed 500ms threshold", "source": "Grafana Alert"},
                {"time": "14:42:00", "event": "First pod restart due to OOMKilled", "source": "Kubernetes"},
                {"time": "14:45:00", "event": "Alert ALT-001 triggered", "source": "AlertManager"}
            ],
            "correlated_events": [
                {"type": "deployment", "description": "payment-service:v2.3.1 deployed 17 minutes ago"},
                {"type": "config_change", "description": "Connection pool size reduced from 100 to 50 in v2.3.1"},
                {"type": "traffic", "description": "Traffic increased 15% (normal for this time)"}
            ],
            "log_analysis": {
                "total_errors": 1247,
                "error_patterns": [
                    {"pattern": "Connection pool exhausted", "count": 892, "percentage": 71.5},
                    {"pattern": "Query timeout after 30000ms", "count": 245, "percentage": 19.6},
                    {"pattern": "OutOfMemoryError: heap space", "count": 110, "percentage": 8.8}
                ],
                "sample_error": "2024-01-30 14:42:18 ERROR PaymentProcessor - Failed to acquire connection from pool after 30000ms"
            },
            "root_cause": {
                "confidence": 0.92,
                "primary_cause": "Database connection pool exhaustion",
                "explanation": "Deployment v2.3.1 reduced connection pool size from 100 to 50. Combined with normal traffic increase, this caused connection starvation.",
                "contributing_factors": ["Connection pool size reduced by 50%", "No connection pool monitoring alerts", "Traffic increased 15%"]
            },
            "recommended_actions": [
                {"priority": 1, "action": "Execute 'clear-connection-pool' runbook", "impact": "Immediate relief", "risk": "low"},
                {"priority": 2, "action": "Rollback to payment-service:v2.3.0", "impact": "Restores original pool settings", "risk": "medium"},
                {"priority": 3, "action": "Scale out payment-service to 6 replicas", "impact": "Distributes load", "risk": "low"}
            ],
            "similar_incidents": [
                {"incident_id": "INC-2024-0115", "similarity": 0.87, "resolution": "Increased connection pool size"},
                {"incident_id": "INC-2023-1203", "similarity": 0.72, "resolution": "Rolled back deployment"}
            ]
        }
    }
    
    investigation = investigations.get(alert_id)
    if not investigation:
        return {"error": f"No investigation data for alert '{alert_id}'", "suggestion": "Use get_active_alerts() to see current alerts."}
    
    return investigation

print("✓ investigate_incident tool defined")

In [ ]:
# Tool 5: Send notification
@tool  
def send_notification(
    channel: Annotated[str, Field(description="Notification channel: 'teams', 'slack', 'pagerduty', or 'email'")],
    message: Annotated[str, Field(description="Message content (markdown supported)")],
    severity: Annotated[str, Field(description="Severity level: 'info', 'warning', or 'critical'")] = "info",
    recipients: Annotated[str, Field(description="Recipients or channel name")] = "sre-oncall"
) -> Dict[str, Any]:
    """
    Send a notification to the specified channel.
    Use this to alert team members about incidents, updates, or completed remediations.
    """
    severity_config = {
        "info": {"color": "#0078D4", "icon": "info-circle", "priority": "low"},
        "warning": {"color": "#FF8C00", "icon": "exclamation-triangle", "priority": "medium"},
        "critical": {"color": "#D13438", "icon": "times-circle", "priority": "high"}
    }
    config = severity_config.get(severity, severity_config["info"])
    
    return {
        "status": "sent",
        "notification": {
            "channel": channel,
            "recipients": recipients,
            "severity": severity,
            "message_preview": message[:200] + "..." if len(message) > 200 else message,
            "formatting": {"color": config["color"], "icon": config["icon"]}
        },
        "delivery": {
            "sent_at": datetime.now().isoformat(),
            "message_id": f"msg-{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "priority": config["priority"]
        }
    }

print("✓ send_notification tool defined")
print("\n✓ All 5 SRE tools defined!")

---

## Step 3: Create the SRE Agent

In [ ]:
# Configuration
PROJECT_ENDPOINT = os.environ.get("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

if not PROJECT_ENDPOINT or PROJECT_ENDPOINT.startswith("https://<"):
    print("⚠️  PROJECT_ENDPOINT not configured in .env file")
    print("   Copy .env.sample to .env and set your Azure AI Foundry endpoint")
    raise ValueError("PROJECT_ENDPOINT not configured")
else:
    print(f"✓ Using endpoint: {PROJECT_ENDPOINT[:50]}...")
    print(f"✓ Using model: {MODEL_DEPLOYMENT}")

In [ ]:
SRE_INSTRUCTIONS = """
You are an Azure SRE Agent assistant. Your role is to help Site Reliability Engineers 
monitor, diagnose, and remediate issues in their Azure infrastructure.

## Your Capabilities
1. **Monitoring**: Query service metrics and check system health
2. **Alerting**: View and triage active alerts across the infrastructure  
3. **Investigation**: Perform root cause analysis on incidents
4. **Remediation**: Execute runbooks to fix issues (always dry-run first!)
5. **Communication**: Send notifications to the team

## Guidelines
- Always check current alerts and metrics before suggesting actions
- When executing runbooks, ALWAYS do a dry-run first to preview changes
- Provide clear explanations of what you find and recommend
- For critical issues, suggest notifying the on-call team
- When investigating, correlate multiple data sources (metrics, logs, deployments)
"""

# Initialize credential and client
credential = DefaultAzureCredential()

chat_client = AzureAIAgentClient(
    project_endpoint=PROJECT_ENDPOINT,
    credential=credential,
    model_deployment_name=MODEL_DEPLOYMENT,
)

# Create the agent with all SRE tools
sre_agent = ChatAgent(
    name="SREAgent",
    instructions=SRE_INSTRUCTIONS,
    chat_client=chat_client,
    tools=[query_service_metrics, get_active_alerts, execute_runbook, investigate_incident, send_notification],
)

print("✓ SRE Agent created with 5 tools:")
print("  • query_service_metrics - Check service health and metrics")
print("  • get_active_alerts - View current alerts")
print("  • execute_runbook - Run remediation playbooks")
print("  • investigate_incident - Root cause analysis")
print("  • send_notification - Alert the team")

---

## Step 4: Test the Agent Locally

In [ ]:
# Test the agent with a simple query
thread = sre_agent.get_new_thread()

print("Query: 'What alerts are currently active?'")
print("=" * 60)

async for update in sre_agent.run_stream("What alerts are currently active?", thread=thread):
    if update.text:
        print(update.text, end="", flush=True)

print("\n")

In [ ]:
# Test with investigation
print("Query: 'Investigate ALT-001 and suggest remediation'")
print("=" * 60)

async for update in sre_agent.run_stream(
    "Investigate the alert ALT-001 and suggest what we should do to fix it", 
    thread=thread
):
    if update.text:
        print(update.text, end="", flush=True)

print("\n")

---

## Step 5: Set Up AG-UI Server

Now let's expose this agent via the **AG-UI protocol** so it can be consumed by rich UI clients.

AG-UI enables:
- **Streaming** agent responses in real-time
- **Rich tool results** rendered as UI components
- **Standardized protocol** - any AG-UI client can connect

In [ ]:
from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

# Create FastAPI app
app = FastAPI(
    title="SRE Agent - AG-UI Server",
    description="Azure SRE Agent exposed via AG-UI protocol",
    version="1.0.0"
)

# Enable CORS for web clients
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, specify allowed origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Register the AG-UI endpoint
add_agent_framework_fastapi_endpoint(app, sre_agent, "/")

print("✓ AG-UI endpoint registered")
print("\nEndpoints:")
print("  POST /  - AG-UI chat endpoint (streaming)")

---

## Step 6: Run the AG-UI Server

Run this cell to start the AG-UI server on port 8888.

**Note:** This cell will block while the server is running. Use "Interrupt Kernel" to stop it.

In [ ]:
import uvicorn
import nest_asyncio

# Allow nested event loops (needed for Jupyter)
nest_asyncio.apply()

print("🚀 Starting AG-UI Server...")
print("=" * 60)
print("Server URL: http://127.0.0.1:8888")
print("")
print("To connect a client:")
print("  1. CopilotKit: Configure agent URL as http://127.0.0.1:8888")
print("  2. CLI Client: python agui_client.py")
print("  3. curl:")
print("")
print('curl -X POST http://127.0.0.1:8888/ \\')
print('  -H "Content-Type: application/json" \\')
print('  -d \'{"messages": [{"role": "user", "content": "What alerts are active?"}]}\'') 
print("")
print("Press Ctrl+C or Interrupt Kernel to stop")
print("=" * 60)

uvicorn.run(app, host="127.0.0.1", port=8888)

---

## Understanding AG-UI Tool Rendering

When the agent calls tools, AG-UI streams structured results that clients render as components:

### Metrics Data → Charts & Gauges
```json
{
  "metrics": {
    "request_rate": {"value": 1250, "unit": "req/min", "trend": "up"},
    "error_rate": {"value": 2.5, "unit": "%", "trend": "up"}
  },
  "time_series": [...]
}
```
→ Rendered as **metric cards**, **line charts**, **trend indicators**

### Alerts Data → Tables & Cards  
```json
{
  "alerts": [{"id": "ALT-001", "severity": "critical", "title": "High Latency"}]
}
```
→ Rendered as **alert cards** with severity badges, **sortable tables**

### Runbook Results → Step Progress
```json
{
  "execution_plan": [
    {"step": 1, "action": "Scale to 0", "status": "completed"},
    {"step": 2, "action": "Clear cache", "status": "in_progress"}
  ]
}
```
→ Rendered as **step-by-step progress**, **status indicators**

---

## Bonus: Extending with MCP

The Agent Framework supports **MCP (Model Context Protocol)** for connecting to external tools. Use `MCPTool` to connect any MCP server:

```python
from agent_framework import ChatAgent, MCPTool

# Connect to an MCP server
mcp_tool = MCPTool(endpoint_uri="https://your-mcp-server.com")

# Use MCP tools in your agent
agent = ChatAgent(
    chat_client=chat_client,
    name="SRE-Agent-with-MCP",
    instructions="You have access to runbook automation tools.",
    tools=[mcp_tool],  # MCP tools are automatically discovered
)
```

This allows you to:
- Connect to existing MCP servers (databases, APIs, etc.)
- Use the same agent with different tool backends
- Dynamically discover available tools

---

## Summary

In this lab, you learned how to:

1. **Create SRE tools** that return structured data suitable for rich rendering
2. **Build an agent** with multiple specialized tools for infrastructure management  
3. **Expose via AG-UI** using FastAPI and `agent-framework-ag-ui`
4. **Understand the protocol** for how tool results map to UI components

### Key Concepts

| Concept | Description |
|---------|-------------|
| **AG-UI Protocol** | Standardized protocol for agent-to-UI communication |
| **Structured Tool Results** | Return rich data structures that clients can render |
| **Streaming** | Real-time updates as the agent processes |
| **MCP Integration** | Connect external tools via Model Context Protocol |

### Next Steps

1. **Build a custom UI** using CopilotKit or your own React components
2. **Add more tools** for your specific infrastructure needs
3. **Integrate with real monitoring** (Azure Monitor, Prometheus, etc.)
4. **Deploy to production** using Azure Container Apps or AKS

---

## References

| Resource | URL |
|----------|-----|
| AG-UI Protocol | https://github.com/ag-ui-protocol/ag-ui |
| CopilotKit | https://copilotkit.ai |
| Microsoft Agent Framework | https://github.com/microsoft/agent-framework |
| Model Context Protocol | https://modelcontextprotocol.io/ |
| Azure SRE Agent | https://learn.microsoft.com/azure/sre-agent/overview |